# Neural Network

http://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py

신경망은 torch.nn패키지를 사용해 구축가능하다.

autograd를 체험해봤다면 모델을 구현하기위해 autograd를 이용해서 변수들을 미분한다. nn.Module은 레이어를 포함하고 forward(input)은 output을 리턴한다.

신경망의 전형적인 학습 과정은 다음과 같다.

* 학습가능한 파라미터를 가진 신경망을 정의한다.
* 데이터셋을 반복적으로 입력한다.
* 신경망을 통해 입력을 계산한다.
* Loss를 계산한다(얼마나 output이 정확한지)
* 신경망 파라미터를 backpropagation한다.
* 신경망의 가중치를 업데이트한다. 일반적으로는 weight = weight - learning_rate * gradient 로 업데이트한다.

![](http://pytorch.org/tutorials/_images/mnist.png)

### 신경망을 정의한다.

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation : y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number.
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()

In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # Conv1's weight

10
torch.Size([6, 1, 5, 5])


In [5]:
input_ = Variable(torch.randn(1,1, 32, 32))
out = net(input_)
print(out)

Variable containing:
-0.0057  0.1085 -0.0623  0.1238  0.0066  0.0385 -0.0665  0.0390 -0.0985  0.0326
[torch.FloatTensor of size 1x10]



Variable containing:
(0 ,0 ,.,.) = 
 -1.9750  1.6966  1.5416  ...   0.6682 -0.2747  0.8042
  0.7253  0.1674 -0.2985  ...   0.7485  0.4294  1.1410
 -0.0499  0.0620  0.3612  ...  -1.4509 -1.3365  0.6629
           ...             ⋱             ...          
 -1.2124  1.5354  0.5902  ...   0.6747 -0.3926  2.4623
  0.0049 -1.0079  0.4455  ...  -1.8563 -0.2620 -0.1813
 -0.2677 -0.7530  0.0013  ...   0.3582 -1.4650  0.4767
[torch.FloatTensor of size 1x1x32x32]

모든 파라미터의 버퍼를 0으로 초기화

In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

torch.nn은 오직 미니-배치 를 지원한다. 전체 torch.nn패키지는 샘플의 미니배치만을 입력으로 지원한다. 싱글 샘플말고.

예를들어, nn.Conv2d는 4D 텐서를 입력받는다.
nSamples * nChannels * Height(row) * Width(col).

만약 싱글 샘풀을 갖고있다면, input.unsqueeze(0)을 써서 배치차원을 추가해주자.

다음으로 넘어가기전에 보충을 한번 하자.

* torch.Tensor - 다차원 array
* autograd.Variable - 텐서를 감싸고 적용된 연산의 순서를 기억한다. Tensor와 같은 API를 갖고 backward()가 있다. 
* nn.Module - 신경망 모듈. GPU로 옮기는 헬퍼함수, 외부저장, 로딩 등.. 파라미터를 캡슐화하는 편한 방식이다. 
* nn.Parameter - Variable 처럼 Module의 attribute로 등록된 파라미터들
* autograd.Function - autograd 연산의 전방/역방 정의의 구현.
최소 싱글 Function노드를 만드는 모든 Variable객체의 연산은 Variable이 만든 함수로 연결되어있고 그 순서를 암호화한다.

지금까지 우린 다음과 같은 사항을 진행해왔다.
* 신경망을 구현
* 입력을 처리하고 역전파 함수를 호출

아직 남은 부분은..
* loss 계산
* 신경망 가중치 업데이트